In [19]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
from src.inception_cifar100 import inceptionv3
from src.temperature_scaling import ModelWithTemperature
import numpy as np
from src.saps import split_data_set, saps_scores, saps_classification, eval_aps


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dict_path = "C:\\Users\\jiayang\\ipynb\\trainedModel\\Inception_CIFAR100.pth"
model = inceptionv3()
model.load_state_dict(torch.load(dict_path, map_location=device, weights_only=True))
model.to(device)

# preprocess the images from CIFAR10
data_transform = transforms.Compose([
    transforms.ToTensor(),         
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  
])

# load data set from CIFAR100
dataset = CIFAR100(root="../../data", train=False, download=True,transform=data_transform)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
#Hyperparameter
lambda_ = 0.13

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, q_hat, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
SAPS Classification, Start!

Running experiment 1/10...
q_hat = 1.0964112997055058
Average Prediction Set Size After APS in runs 1: 3.1784
Average Coverage Rate in runs 1: 0.907

Running experiment 2/10...
q_hat = 1.0756803274154667
Average Prediction Set Size After APS in runs 2: 3.0276
Average Coverage Rate in runs 2: 0.8994

Running experiment 3/10...
q_hat = 1.0740899562835697
Average Prediction Set Size After APS in runs 3: 3.0244
Average Coverage Rate in runs 3: 0.8952

Running experiment 4/10...
q_hat = 1.074631273746491
Average Prediction Set Size After APS in runs 4: 3.0036
Average Coverage Rate in runs 4: 0.8986

Running experiment 5/10...
q_hat = 1.0885197997093201
Average Prediction Set Size After APS in runs 5: 3.1088
Average Coverage Rate in runs 5: 0.9028

Running experiment 6/10...
q_hat = 1.0753330588340764
Average Prediction Set Size After APS in runs 6: 3.0534
Average Coverage Rate in runs 6: 0.8976

Running experiment 7/10...
q_

#  Result

From the above test, following results can be collected :
- Final Average Prediction Set Size: 3.06/100
- Final Average Coverage: 89.91% ($\alpha$=0.1)